In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import r2_score, mean_squared_error
import optuna
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
import joblib
import random

C:\Users\ucbab\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

data = pd.read_excel("C:/Users/ucbab/OneDrive/Documents/Pasc-Data.xlsx")

In [3]:
x= data.drop(columns=['Biodiesel yield']).values
y=data['Biodiesel yield'].values

In [4]:
scaler_x = StandardScaler()
scaler_y= StandardScaler()
x = scaler_x.fit_transform(x)
y=scaler_y.fit_transform(y.reshape(-1,1))


In [61]:
SEED = 86
np.random.seed(SEED)


In [62]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=SEED)


In [63]:
best_score = -np.inf
best_model = None

def objective(trial): 
    global best_score, best_model

    n_estimators = trial.suggest_int("n_estimators", 50, 500, step=50)  # Number of trees
    max_depth = trial.suggest_int("max_depth", 5, 50)  # Maximum depth of each tree
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)  # Minimum number of samples required to split an internal node
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)  # Minimum number of samples required to be at a leaf node
    max_features = trial.suggest_categorical("max_features", [ "sqrt", "log2"]) 

    

    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=SEED,
        n_jobs=-1
    )

    kf = KFold(n_splits=5, shuffle=True, random_state=SEED)
    scores = cross_val_score(model, x_train, y_train, cv=kf, scoring="r2")
    mean_score = np.mean(scores)

    if mean_score > best_score:
        best_score = mean_score
        best_model = model
        model.fit(x_train, y_train)
        joblib.dump(model, 'best_rf_bio_model.pkl')

    return mean_score


In [64]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("Best R² score from CV:", best_score)
print("Best hyperparameters:", study.best_params)
 

[I 2025-05-20 20:39:46,945] A new study created in memory with name: no-name-7208d18c-9474-44b5-9a27-31b200496e62
  0%|          | 0/50 [00:00<?, ?it/s]C:\Users\ucbab\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\ucbab\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\ucbab\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Pytho

[I 2025-05-20 20:39:53,893] Trial 0 finished with value: -0.20755965419537117 and parameters: {'n_estimators': 400, 'max_depth': 25, 'min_samples_split': 19, 'min_samples_leaf': 5, 'max_features': 'sqrt'}. Best is trial 0 with value: -0.20755965419537117.


C:\Users\ucbab\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\ucbab\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\ucbab\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y 

[I 2025-05-20 20:39:58,429] Trial 1 finished with value: -0.21002008466532202 and parameters: {'n_estimators': 350, 'max_depth': 32, 'min_samples_split': 17, 'min_samples_leaf': 20, 'max_features': 'log2'}. Best is trial 0 with value: -0.20755965419537117.


C:\Users\ucbab\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\ucbab\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\ucbab\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y 

...

[I 2025-05-20 20:43:26,256] Trial 47 finished with value: 0.36209908996153506 and parameters: {'n_estimators': 250, 'max_depth': 23, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'log2'}. Best is trial 28 with value: 0.45631089093963884.


C:\Users\ucbab\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\ucbab\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\ucbab\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y 

[I 2025-05-20 20:43:36,787] Trial 48 finished with value: 0.3600654594055273 and parameters: {'n_estimators': 400, 'max_depth': 40, 'min_samples_split': 7, 'min_samples_leaf': 1, 'max_features': 'sqrt'}. Best is trial 28 with value: 0.45631089093963884.


C:\Users\ucbab\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\ucbab\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\ucbab\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y 

[I 2025-05-20 20:43:44,197] Trial 49 finished with value: 0.27634112564039565 and parameters: {'n_estimators': 300, 'max_depth': 21, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': 'log2'}. Best is trial 28 with value: 0.45631089093963884.
Best R² score from CV: 0.45631089093963884
Best hyperparameters: {'n_estimators': 350, 'max_depth': 24, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 'sqrt'}


In [65]:

# Load the best saved model
rf_model = joblib.load('best_rf_bio_model.pkl')

# Predict
y_train_pred = rf_model.predict(x_train)
y_test_pred = rf_model.predict(x_test)

# R² Scores
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"Train R²: {train_r2:.4f}")
print(f"Test R²: {test_r2:.4f}")


Train R²: 0.8980
Test R²: 0.8566


In [5]:


print("scikit-learn version:", sklearn.__version__)

scikit-learn version: 1.5.2


In [66]:
# Display best hyperparameters
best_params = study.best_params
print("Best Hyperparameters:", best_params)
print("Best validation R² Score from Optuna:", study.best_value)

Best Hyperparameters: {'n_estimators': 350, 'max_depth': 24, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 'sqrt'}
Best validation R² Score from Optuna: 0.45631089093963884


In [68]:
# Predict on the full dataset
y_pred_full = best_model.predict(x)

# Calculate R² score for the full dataset
r2_full = r2_score(y, y_pred_full)
print(f'R² Score for the full dataset: {r2_full:.4f}')

R² Score for the full dataset: 0.8895


In [69]:
mse_full = mean_squared_error(y, y_pred_full)
print(f'Mean Squared Error for the full dataset: {mse_full:.4f}')

Mean Squared Error for the full dataset: 0.1105


In [70]:
rmse_full = np.sqrt(mean_squared_error(y, y_pred_full))
print(f'Root Mean Squared Error for the full dataset: {rmse_full:.4f}')

Root Mean Squared Error for the full dataset: 0.3324


In [82]:
aad_full = np.mean(np.abs(y.flatten() - y_pred_full))
print(f'Average Absolute Deviation (AAD) for the full dataset: {aad_full:.4f}')

Average Absolute Deviation (AAD) for the full dataset: 0.2683


In [83]:
# Calculate AIC for the full dataset
n = len(y)  # number of samples
k = len(best_model.get_params())  # number of model parameters
rss = np.sum((y.flatten() - y_pred_full) ** 2)  # residual sum of squares

aic_full = n * np.log(rss / n) + 2 * k
print(f'AIC for the full dataset: {aic_full:.4f}')

AIC for the full dataset: -30.0794


In [76]:
y_pred_full

array([-0.70181445, -0.95545902, -0.67362357, -0.79931051,  0.83581334,
        0.314498  ,  1.09712456,  0.97838037, -0.91108629, -1.24973945,
       -0.83332491, -0.93985051,  0.62308256, -0.19226728,  0.99750466,
        0.76622387,  0.30746697, -0.27657691, -0.12562406,  0.59474771,
       -1.33408403,  1.35099729, -0.04577994, -0.02176494,  0.42830204,
        0.42830204,  0.42830204,  0.42830204,  0.42830204,  0.42830204])

In [84]:
# Inverse transform the scaled features and yields
features_unscaled = scaler_x.inverse_transform(x)
actual_yield_unscaled = scaler_y.inverse_transform(y).flatten()
predicted_yield_unscaled = scaler_y.inverse_transform(y_pred_full.reshape(-1, 1)).flatten()

# Create the dataframe
df_full_unscaled = pd.DataFrame(
    features_unscaled,
    columns=['Temperature', 'Catalyst dosage', 'MeOH-Oil ratio', 'Time']
)
df_full_unscaled['Actual Biodiesel Yield'] = actual_yield_unscaled
df_full_unscaled['Predicted Biodiesel Yield'] = predicted_yield_unscaled

df_full_unscaled

,Temperature,Catalyst dosage,MeOH-Oil ratio,Time,Actual Biodiesel Yield,Predicted Biodiesel Yield
0,60.0,2.0,12.0,1.38,47.0264,46.527824
1,80.0,2.0,12.0,1.38,43.5159,41.499386
2,60.0,4.0,12.0,1.38,33.2807,47.086701
3,80.0,4.0,12.0,1.38,40.5441,44.594990
4,60.0,2.0,24.0,1.38,84.0970,77.010896
5,80.0,2.0,24.0,1.38,69.8490,66.675955
6,60.0,4.0,24.0,1.38,91.5153,82.191323
7,80.0,4.0,24.0,1.38,88.0412,79.837250
8,60.0,2.0,12.0,3.13,42.5897,42.379064
9,80.0,2.0,12.0,3.13,28.7631,35.665353
